<a href="https://colab.research.google.com/github/RomanTomz/RomeHeatmap/blob/master/Incidenti_Roma_2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
! pip install geojson
! pip install geopandas


In [102]:
import pandas as pd
import glob
import geojson
import plotly
import geopandas as gpd
import json
import plotly.graph_objects as go
import altair as alt
import plotly.express as px 


In [103]:
df = pd.concat([pd.read_csv(f,  sep=';', encoding='latin1') for f in glob.glob('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/csv_incidenti_*.csv')])


In [104]:
df.drop_duplicates(subset='Protocollo', inplace=True)
df.DataOraIncidente = pd.to_datetime(df.DataOraIncidente)
df.sort_values('DataOraIncidente', inplace=True)

In [105]:
df.head(3)

,Protocollo,Gruppo,DataOraIncidente,Localizzazione1,STRADA1,Localizzazione2,STRADA2,Strada02,Chilometrica,DaSpecificare,NaturaIncidente,particolaritastrade,TipoStrada,FondoStradale,Pavimentazione,Segnaletica,CondizioneAtmosferica,Traffico,Visibilita,Illuminazione,NUM_FERITI,NUM_RISERVATA,NUM_MORTI,NUM_ILLESI,Longitudine,Latitudine,Confermato,Progressivo,TipoVeicolo,StatoVeicolo,TipoPersona,Sesso,Tipolesione,Deceduto,DecedutoDopo,CinturaCascoUtilizzato,Airbag,Unnamed: 37,Longitude,Latitude
136,4160143,8,2019-01-01 01:10:00,Strada Urbana,VIA CASILINA,in prossimità,NaN,del civico n.,1765,NaN,Tamponamento,Rettilineo,Una carreggiata a doppio senso,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Intenso,Buona,Ore Diurne,1,0,0,2,12.6703,41.8640,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,F,Rifiuta cure immediate,0.0,NON DECEDUTO,Non accertato,Inesploso,NaN,NaN,NaN
75,4160005,1,2019-01-01 01:30:00,Strada Urbana,VIA DI MONTE TESTACCIO,in corrispondenza,NaN,del civico n.,39,NaN,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Buona,Sufficiente,0,0,0,3,12.4741,41.8757,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Illeso,0.0,NaN,Non accertato,Inesploso,NaN,NaN,NaN
3018,4181959,1,2019-01-01 02:00:00,Strada Urbana,PIAZZA DI S. CROCE IN GERUSALEMME,da specificare,NaN,NaN,NaN,impianto semaforico prima di Carlo Felice,Tamponamento,Rettilineo,Più di due carreggiate,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Sufficiente,Insufficiente,1,0,0,2,12.5147,41.8888,-1,1.0,Motociclo a solo,In marcia / fermata / arresto,Conducente,M,Rimandato,0.0,NON DECEDUTO,NaN,NaN,NaN,NaN,NaN


In [106]:
viario = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/viario.csv', encoding='latin1', sep=';', header=None, usecols=[0,1,2])

In [107]:
viario.rename(columns={0:'municipio',1:'codice',2:'denominazione'}, inplace=True)

In [108]:
viario.denominazione = viario.denominazione.str.strip()

In [109]:
viario

,municipio,codice,denominazione
0,19,2357.0,VIA ABANO TERME
1,10,1126.0,VIA ABASCANTO
2,16,2530.0,VIA ABATE UGONE
3,20,2440.0,VIA ABBADIA SAN SALVATORE
4,19,2392.0,VIA ABBIATEGRASSO
...,...,...,...
30290,13,2048.0,VIA LUIGI CARDINI
30291,13,2048.0,VIA PAOLA ZANCANI MONTUORO
30292,8,627.0,VIA SAN PIETRO AVELLANA
30293,10,1191.0,VIA ROZZANO


In [110]:
df.STRADA1 = df.STRADA1.str.strip()

In [111]:
merged = pd.merge(df, viario, how='left', left_on='STRADA1', right_on='denominazione').drop_duplicates(subset='Protocollo').reset_index(drop=True)
merged.head(3)

,Protocollo,Gruppo,DataOraIncidente,Localizzazione1,STRADA1,Localizzazione2,STRADA2,Strada02,Chilometrica,DaSpecificare,NaturaIncidente,particolaritastrade,TipoStrada,FondoStradale,Pavimentazione,Segnaletica,CondizioneAtmosferica,Traffico,Visibilita,Illuminazione,NUM_FERITI,NUM_RISERVATA,NUM_MORTI,NUM_ILLESI,Longitudine,Latitudine,Confermato,Progressivo,TipoVeicolo,StatoVeicolo,TipoPersona,Sesso,Tipolesione,Deceduto,DecedutoDopo,CinturaCascoUtilizzato,Airbag,Unnamed: 37,Longitude,Latitude,municipio,codice,denominazione
0,4160143,8,2019-01-01 01:10:00,Strada Urbana,VIA CASILINA,in prossimità,NaN,del civico n.,1765,NaN,Tamponamento,Rettilineo,Una carreggiata a doppio senso,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Intenso,Buona,Ore Diurne,1,0,0,2,12.6703,41.8640,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,F,Rifiuta cure immediate,0.0,NON DECEDUTO,Non accertato,Inesploso,NaN,NaN,NaN,9,854.0,VIA CASILINA
1,4160005,1,2019-01-01 01:30:00,Strada Urbana,VIA DI MONTE TESTACCIO,in corrispondenza,NaN,del civico n.,39,NaN,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Buona,Sufficiente,0,0,0,3,12.4741,41.8757,-1,1.0,Autovettura privata,In marcia / fermata / arresto,Conducente,M,Illeso,0.0,NaN,Non accertato,Inesploso,NaN,NaN,NaN,1,1225.0,VIA DI MONTE TESTACCIO
2,4181959,1,2019-01-01 02:00:00,Strada Urbana,PIAZZA DI S. CROCE IN GERUSALEMME,da specificare,NaN,NaN,NaN,impianto semaforico prima di Carlo Felice,Tamponamento,Rettilineo,Più di due carreggiate,Asciutto,Asfaltata,Verticale ed orizzontale,Sereno,Normale,Sufficiente,Insufficiente,1,0,0,2,12.5147,41.8888,-1,1.0,Motociclo a solo,In marcia / fermata / arresto,Conducente,M,Rimandato,0.0,NON DECEDUTO,NaN,NaN,NaN,NaN,NaN,1,16.0,PIAZZA DI S. CROCE IN GERUSALEMME


In [112]:
merged.columns

Index(['Protocollo', 'Gruppo', 'DataOraIncidente', 'Localizzazione1',
       'STRADA1', 'Localizzazione2', 'STRADA2', 'Strada02', 'Chilometrica',
       'DaSpecificare', 'NaturaIncidente', 'particolaritastrade', 'TipoStrada',
       'FondoStradale', 'Pavimentazione', 'Segnaletica',
       'CondizioneAtmosferica', 'Traffico', 'Visibilita', 'Illuminazione',
       'NUM_FERITI', 'NUM_RISERVATA', 'NUM_MORTI', 'NUM_ILLESI', 'Longitudine',
       'Latitudine', 'Confermato', 'Progressivo', 'TipoVeicolo',
       'StatoVeicolo', 'TipoPersona', 'Sesso', 'Tipolesione', 'Deceduto',
       'DecedutoDopo', 'CinturaCascoUtilizzato', 'Airbag', 'Unnamed: 37',
       'Longitude', 'Latitude', 'municipio', 'codice', 'denominazione'],
      dtype='object')

In [113]:
merged = merged[['Protocollo','DataOraIncidente','Localizzazione1','STRADA1','NaturaIncidente','particolaritastrade','TipoStrada','FondoStradale','Segnaletica','CondizioneAtmosferica','Traffico','Illuminazione','NUM_FERITI',
        'NUM_MORTI','NUM_ILLESI','Longitudine','Latitudine','TipoVeicolo','TipoPersona','Sesso','Tipolesione','DecedutoDopo','municipio']]

In [114]:
with open('/content/drive/My Drive/Colab Notebooks/Roma/Inc_Roma_2019/ROMA_MUNICIPI.geojson', encoding='latin1') as f:
    municipi = json.load(f)

In [115]:
for feature in municipi['features']:
  municipi['id'] = feature['properties']['MUNICIPIO']

In [116]:
mun_feriti = merged.groupby('municipio')[['NUM_FERITI']].sum().reset_index(drop=False)

In [117]:
mun_feriti

,municipio,NUM_FERITI
0,1,1629
1,10,609
2,11,711
3,12,705
4,13,763
5,15,404
6,16,684
7,17,726
8,18,866
9,19,462


### Analisi

In [118]:
merged.head()

,Protocollo,DataOraIncidente,Localizzazione1,STRADA1,NaturaIncidente,particolaritastrade,TipoStrada,FondoStradale,Segnaletica,CondizioneAtmosferica,Traffico,Illuminazione,NUM_FERITI,NUM_MORTI,NUM_ILLESI,Longitudine,Latitudine,TipoVeicolo,TipoPersona,Sesso,Tipolesione,DecedutoDopo,municipio
0,4160143,2019-01-01 01:10:00,Strada Urbana,VIA CASILINA,Tamponamento,Rettilineo,Una carreggiata a doppio senso,Asciutto,Verticale ed orizzontale,Sereno,Intenso,Ore Diurne,1,0,2,12.6703,41.8640,Autovettura privata,Conducente,F,Rifiuta cure immediate,NON DECEDUTO,9
1,4160005,2019-01-01 01:30:00,Strada Urbana,VIA DI MONTE TESTACCIO,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Verticale ed orizzontale,Sereno,Normale,Sufficiente,0,0,3,12.4741,41.8757,Autovettura privata,Conducente,M,Illeso,NaN,1
2,4181959,2019-01-01 02:00:00,Strada Urbana,PIAZZA DI S. CROCE IN GERUSALEMME,Tamponamento,Rettilineo,Più di due carreggiate,Asciutto,Verticale ed orizzontale,Sereno,Normale,Insufficiente,1,0,2,12.5147,41.8888,Motociclo a solo,Conducente,M,Rimandato,NON DECEDUTO,1
3,4160012,2019-01-01 02:00:00,Strada Urbana,LUNGOTEVERE TOR DI NONA,Scontro laterale fra veicoli in marcia,Rettilineo,Una carreggiata a senso unico di marcia,Asciutto,Verticale ed orizzontale,Sereno,Normale,Sufficiente,0,0,9,12.4692,41.9015,Autovettura privata,Conducente,M,Illeso,NaN,1
4,4159601,2019-01-01 02:20:00,Strada Urbana,VIALE CORTINA D'AMPEZZO,Scontro frontale fra veicoli in marcia,Curva a visuale libera,Una carreggiata a senso unico di marcia,Asciutto,Verticale ed orizzontale,Sereno,Scarso,Sufficiente,4,0,0,12.4393,41.9465,Autovettura privata,Conducente,M,Ricoverato,NON DECEDUTO,20


In [119]:
hourly = merged.groupby(merged.DataOraIncidente.dt.hour)[['NUM_FERITI']].sum()

In [120]:
px.bar(hourly, x=hourly.index, y='NUM_FERITI')

In [160]:
fig = px.density_mapbox(merged, lat='Latitudine', lon='Longitudine', z=merged.Protocollo.sum(), radius=10,
                        center=dict(lat=41.9028, lon=12.4964), zoom=11.5,
                        mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

fig.show()

In [139]:
merged.groupby('municipio')['Protocollo'].count()

municipio
1     3347
10    1292
11    1567
12    1590
13    1745
15    1034
16    1453
17    1526
18    1538
19    1074
2     2503
20    1581
3     1027
4     1341
5     1124
6     1270
7      859
8     1674
9     2161
Name: Protocollo, dtype: int64